##PROBLEM 3 : Implement PCA on MNIST
Repeat PB2 exercises on MNIST (D=5 and D=20) with your own PCA implementation. You can use any built-in library/package/API for : matrix storage/multiplication, covariance computation, eigenvalue or SVD decomposition, etc. Matlab is probably the easiest language for implementing PCA due to its excellent linear algebra support.

In [ ]:
!pip install idx2numpy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import idx2numpy
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


###PCA implementation

In [ ]:
class PCA_implementation:
  def __init__(self,n_components):
    self.n_components = n_components
    self.covariance_matrix = None
    self.eigenvalues = None
    self.eigenvectors = None

  def fit(self,X):
    # Center the data by subtracting mean from each data point
    mean = np.mean(X, axis=0)
    X_centered = X - mean

    # Get Covariance Matrix (Cov) (dxd)
    self.covariance_matrix = 1/(X.shape[0]-1) * X_centered.T @ X_centered

    # Get Eigen Vectors and Eigen Values from Cov
    eigenvalues, eigenvectors = np.linalg.eigh(self.covariance_matrix)

    # Projection Matrix P = Eigen Vectors * Sqrt(Eigen Values) = (dxd)
    descending_indices = np.arange(X.shape[1]-1,-1,-1)
    eigenvalues = eigenvalues[descending_indices]
    eigenvectors = eigenvectors[:, descending_indices]
    #eigenvalues = np.clip(eigenvalues, a_min=0, a_max=None)

    self.eigenvalues = eigenvalues
    self.eigenvectors = eigenvectors[:,:self.n_components]

    return self

  def transform(self,X):
    # Return Projected k Components : X(nxd) . P_(dxk) = X_(nxk)
    # NOTE:
    # sklearn implementation of PCA does not calculate projection matrix like this
    # projection_matrix = eigenvectors @ np.diag(np.sqrt(eigenvalues))
    mean = np.mean(X, axis=0)
    X_centered = X - mean
    return X_centered @ self.eigenvectors


  def fit_transform(self,X):
    self.fit(X)
    return self.transform(X)


In [ ]:
# TESTING by simple eg
# points on line y = x+1
X = np.array([[1, 2], [2, 3], [3, 4],[4,5],[5,6]])

pca_ = PCA_implementation(n_components=1)
print('My PCA implementation result: \n')
print(pca_.fit_transform(X),'\n')

from sklearn.decomposition import PCA
print('PCA lib result: \n')
pca = PCA(n_components=1)
print(pca.fit_transform(X))


My PCA implementation result: 

[[-2.82842712]
 [-1.41421356]
 [ 0.        ]
 [ 1.41421356]
 [ 2.82842712]] 

PCA lib result: 

[[ 2.82842712]
 [ 1.41421356]
 [-0.        ]
 [-1.41421356]
 [-2.82842712]]


###A) For MNIST dataset, run a PCA-library to get data on D=5 features. Rerun the classification tasks from PB1, compare testing performance with the one from PB1. Then repeat this exercise for D=20

### Preprocessing MNIST

In [ ]:
# MNIST dataset
mnist_images = idx2numpy.convert_from_file("/content/drive/MyDrive/USML/HW 3A/MNIST/train-images.idx3-ubyte")
mnist_images_data = mnist_images.copy()
mnist_labels = idx2numpy.convert_from_file("/content/drive/MyDrive/USML/HW 3A/MNIST/train-labels.idx1-ubyte")
mnist_images_data = mnist_images_data.reshape(mnist_images.shape[0], -1)
mnist_images_data = mnist_images_data / 255.

mnist_images_test = idx2numpy.convert_from_file("/content/drive/MyDrive/USML/HW 3A/MNIST/t10k-images-idx3-ubyte")
mnist_images_test_data = mnist_images_test.copy()
mnist_labels_test = idx2numpy.convert_from_file("/content/drive/MyDrive/USML/HW 3A/MNIST/t10k-labels-idx1-ubyte")
mnist_images_test_data = mnist_images_test_data.reshape(mnist_images_test.shape[0], -1)
mnist_images_test_data = mnist_images_test_data / 255.

In [ ]:
# PCA

pca_5 = PCA_implementation(n_components=5)
pca_20 = PCA_implementation(n_components=20)

# training data
mnist_pca_5 = pca_5.fit_transform(mnist_images_data)
mnist_pca_20 = pca_20.fit_transform(mnist_images_data)

# test data
mnist_test_pca_5 = pca_5.transform(mnist_images_test_data)
mnist_test_pca_20 = pca_20.transform(mnist_images_test_data)

###L2-reg Logistic Regression on MNIST

In [ ]:
log_reg_mnist_5 = LogisticRegression(penalty='l2', solver='lbfgs', max_iter=5000, multi_class='multinomial')
log_reg_mnist_20 = LogisticRegression(penalty='l2', solver='lbfgs', max_iter=5000, multi_class='multinomial')

In [ ]:
log_reg_mnist_5.fit(mnist_pca_5, mnist_labels)
log_reg_mnist_20.fit(mnist_pca_20, mnist_labels)

LogisticRegression(max_iter=5000, multi_class='multinomial')

#### Classification performance

In [ ]:
# Testing performance from PB1

Accuracy: 0.9179
Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.97      0.96       980
           1       0.97      0.98      0.97      1135
           2       0.91      0.89      0.90      1032
           3       0.89      0.92      0.90      1010
           4       0.91      0.92      0.92       982
           5       0.90      0.86      0.88       892
           6       0.94      0.95      0.94       958
           7       0.93      0.91      0.92      1028
           8       0.86      0.88      0.87       974
           9       0.89      0.89      0.89      1009

    accuracy                           0.92     10000
   macro avg       0.92      0.92      0.92     10000
weighted avg       0.92      0.92      0.92     10000



In [ ]:
# Testing performance from PB2

Accuracy: 0.6871
Classification Report:
              precision    recall  f1-score   support

           0       0.77      0.83      0.80       980
           1       0.89      0.94      0.92      1135
           2       0.71      0.66      0.68      1032
           3       0.59      0.75      0.66      1010
           4       0.63      0.64      0.63       982
           5       0.41      0.27      0.33       892
           6       0.85      0.85      0.85       958
           7       0.75      0.79      0.77      1028
           8       0.50      0.42      0.45       974
           9       0.60      0.64      0.62      1009

    accuracy                           0.69     10000
   macro avg       0.67      0.68      0.67     10000
weighted avg       0.68      0.69      0.68     10000



In [ ]:
# Testing performance for D=5
mnist_pred = log_reg_mnist_5.predict(mnist_test_pca_5)
accuracy = accuracy_score(mnist_labels_test, mnist_pred)
print(f"Accuracy: {accuracy:.4f}")

class_report = classification_report(mnist_labels_test, mnist_pred)
print("Classification Report:")
print(class_report)

Accuracy: 0.6858
Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.89      0.87       980
           1       0.85      0.94      0.90      1135
           2       0.63      0.53      0.58      1032
           3       0.65      0.71      0.68      1010
           4       0.62      0.68      0.65       982
           5       0.61      0.53      0.57       892
           6       0.63      0.67      0.65       958
           7       0.71      0.75      0.73      1028
           8       0.65      0.61      0.63       974
           9       0.58      0.50      0.53      1009

    accuracy                           0.69     10000
   macro avg       0.68      0.68      0.68     10000
weighted avg       0.68      0.69      0.68     10000



In [ ]:
# testing performance from PB2

Accuracy: 0.8792
Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.95      0.94       980
           1       0.95      0.97      0.96      1135
           2       0.89      0.84      0.87      1032
           3       0.85      0.87      0.86      1010
           4       0.88      0.89      0.88       982
           5       0.84      0.79      0.81       892
           6       0.91      0.92      0.92       958
           7       0.90      0.86      0.88      1028
           8       0.82      0.83      0.83       974
           9       0.82      0.85      0.83      1009

    accuracy                           0.88     10000
   macro avg       0.88      0.88      0.88     10000
weighted avg       0.88      0.88      0.88     10000



In [ ]:
# Testing performance for D=20
mnist_pred = log_reg_mnist_20.predict(mnist_test_pca_20)
accuracy = accuracy_score(mnist_labels_test, mnist_pred)
print(f"Accuracy: {accuracy:.4f}")

# Classification report
class_report = classification_report(mnist_labels_test, mnist_pred)
print("Classification Report:")
print(class_report)

Accuracy: 0.8796
Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.96      0.94       980
           1       0.95      0.97      0.96      1135
           2       0.87      0.84      0.85      1032
           3       0.86      0.87      0.86      1010
           4       0.86      0.89      0.88       982
           5       0.81      0.81      0.81       892
           6       0.91      0.91      0.91       958
           7       0.91      0.89      0.90      1028
           8       0.85      0.80      0.82       974
           9       0.83      0.84      0.84      1009

    accuracy                           0.88     10000
   macro avg       0.88      0.88      0.88     10000
weighted avg       0.88      0.88      0.88     10000



###Decision Tree on MNIST

In [ ]:
tree_clf_mnist_5 = DecisionTreeClassifier()
tree_clf_mnist_20 = DecisionTreeClassifier()

In [ ]:
tree_clf_mnist_5.fit(mnist_pca_5, mnist_labels)
tree_clf_mnist_20.fit(mnist_pca_20, mnist_labels)

DecisionTreeClassifier()

#### Classification performance

In [ ]:
# Testing performance from PB1

Accuracy: 0.8916
Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.94      0.93       980
           1       0.94      0.96      0.95      1135
           2       0.88      0.88      0.88      1032
           3       0.84      0.86      0.85      1010
           4       0.89      0.90      0.90       982
           5       0.87      0.84      0.85       892
           6       0.91      0.91      0.91       958
           7       0.93      0.91      0.92      1028
           8       0.83      0.82      0.83       974
           9       0.88      0.88      0.88      1009

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000



In [ ]:
# Testing performance PB2

Accuracy: 0.7026
Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.83      0.82       980
           1       0.94      0.95      0.95      1135
           2       0.78      0.79      0.79      1032
           3       0.64      0.64      0.64      1010
           4       0.62      0.61      0.61       982
           5       0.46      0.47      0.46       892
           6       0.88      0.86      0.87       958
           7       0.75      0.77      0.76      1028
           8       0.46      0.45      0.45       974
           9       0.62      0.59      0.61      1009

    accuracy                           0.70     10000
   macro avg       0.70      0.70      0.70     10000
weighted avg       0.70      0.70      0.70     10000



In [ ]:
mnist_pred_dt = tree_clf_mnist_5.predict(mnist_test_pca_5)
accuracy = accuracy_score(mnist_labels_test, mnist_pred_dt)
print(f"Accuracy: {accuracy:.4f}")

# Classification report
class_report = classification_report(mnist_labels_test, mnist_pred_dt)
print("Classification Report:")
print(class_report)

Accuracy: 0.6781
Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.85      0.84       980
           1       0.93      0.94      0.93      1135
           2       0.64      0.64      0.64      1032
           3       0.63      0.68      0.65      1010
           4       0.59      0.58      0.59       982
           5       0.63      0.61      0.62       892
           6       0.66      0.63      0.65       958
           7       0.69      0.69      0.69      1028
           8       0.61      0.57      0.59       974
           9       0.53      0.54      0.53      1009

    accuracy                           0.68     10000
   macro avg       0.67      0.67      0.67     10000
weighted avg       0.68      0.68      0.68     10000



In [ ]:
# Testing performance PB2

Accuracy: 0.8476
Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.92      0.91       980
           1       0.96      0.95      0.96      1135
           2       0.84      0.83      0.83      1032
           3       0.80      0.80      0.80      1010
           4       0.82      0.83      0.83       982
           5       0.79      0.79      0.79       892
           6       0.94      0.90      0.92       958
           7       0.85      0.87      0.86      1028
           8       0.77      0.78      0.77       974
           9       0.79      0.78      0.79      1009

    accuracy                           0.85     10000
   macro avg       0.85      0.85      0.85     10000
weighted avg       0.85      0.85      0.85     10000



In [ ]:
mnist_pred_dt = tree_clf_mnist_20.predict(mnist_test_pca_20)
accuracy = accuracy_score(mnist_labels_test, mnist_pred_dt)
print(f"Accuracy: {accuracy:.4f}")

# Classification report
class_report = classification_report(mnist_labels_test, mnist_pred_dt)
print("Classification Report:")
print(class_report)

Accuracy: 0.8475
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.90      0.90       980
           1       0.94      0.97      0.95      1135
           2       0.86      0.83      0.85      1032
           3       0.82      0.82      0.82      1010
           4       0.82      0.84      0.83       982
           5       0.77      0.76      0.77       892
           6       0.91      0.90      0.90       958
           7       0.88      0.86      0.87      1028
           8       0.79      0.77      0.78       974
           9       0.78      0.80      0.79      1009

    accuracy                           0.85     10000
   macro avg       0.85      0.85      0.85     10000
weighted avg       0.85      0.85      0.85     10000



###B) Run PCA library on Spambase and repeat one of the classification algorithms. What is the smallest D (number of PCA dimensions) you need to get a comparable test result?

###Preprocessing Spambase

In [ ]:
# Spambase dataset
spambase_dataset = np.loadtxt('/content/drive/MyDrive/USML/HW 3A/spambase/spambase.data', delimiter=',')
spambase_labels = spambase_dataset[:,57]
spambase_data = spambase_dataset[:,:57]

spambase_train_data, spambase_test_data, spambase_train_lbl, spambase_test_lbl = train_test_split(spambase_data, spambase_labels, test_size=0.2, random_state=42)

# Specify the path to your file
file_path = '/content/drive/MyDrive/USML/HW 3A/spambase/features'

# Open the file for reading
with open(file_path, 'r') as file:
    # Read each line in the file
    lines = file.readlines()

target_names = [string.strip() for string in lines]

In [ ]:
# PCA
D = 30
pca = PCA_implementation(n_components=D)
spambase_pca_train = pca.fit_transform(spambase_train_data)

# test data
spambase_pca_test = pca.transform(spambase_test_data)

###L2-reg Logistic Regression on Spambase

In [ ]:
log_reg_spambase = LogisticRegression(penalty='l2', solver='lbfgs', max_iter=5000)
log_reg_spambase.fit(spambase_pca_train, spambase_train_lbl)

LogisticRegression(max_iter=5000)

#### Classification performance

In [ ]:
# Performace on PB1 (57 features)

Accuracy: 0.9218
Classification Report:
              precision    recall  f1-score   support

         0.0       0.92      0.95      0.93       531
         1.0       0.93      0.88      0.91       390

    accuracy                           0.92       921
   macro avg       0.92      0.92      0.92       921
weighted avg       0.92      0.92      0.92       921



In [ ]:
# Testing performace PB2

Number of features: 30
Accuracy: 0.9131
Classification Report:
              precision    recall  f1-score   support

         0.0       0.90      0.96      0.93       531
         1.0       0.94      0.85      0.89       390

    accuracy                           0.91       921
   macro avg       0.92      0.91      0.91       921
weighted avg       0.91      0.91      0.91       921



In [ ]:
print(f"Number of features: {D}")
spambase_pred = log_reg_spambase.predict(spambase_pca_test)
accuracy = accuracy_score(spambase_test_lbl, spambase_pred)
print(f"Accuracy: {accuracy:.4f}")

# Classification report
class_report = classification_report(spambase_test_lbl, spambase_pred)
print("Classification Report:")
print(class_report)

Number of features: 30
Accuracy: 0.8599
Classification Report:
              precision    recall  f1-score   support

         0.0       0.82      0.97      0.89       531
         1.0       0.94      0.71      0.81       390

    accuracy                           0.86       921
   macro avg       0.88      0.84      0.85       921
weighted avg       0.87      0.86      0.86       921

